In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.20.0


In [2]:
#STEP 1: Import Libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [6]:
#STEP 2: Load the Text File
# Load text data
# STEP 2: Load the Text File
with open(r"C:\Users\Lenovo\OneDrive\Desktop\project _file\Handwriten.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Repeat text to increase training data
text = text * 50

print("Total characters:", len(text))
print(text[:200])

Total characters: 44800
dear diary
today i woke up late
my eyes were still heavy
and my handwriting feels slow and uneven

i tried to write neatly
but my hand moved faster than my thoughts
some letters became big
some became


In [7]:
import numpy as np
import tensorflow as tf

# Unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)

char2idx = {c: i for i, c in enumerate(chars)}
idx2char = np.array(chars)

print("Total unique characters:", vocab_size)

# Encode full text
encoded_text = np.array([char2idx[c] for c in text])


Total unique characters: 26


In [8]:
SEQ_LENGTH = 40
BATCH_SIZE = 32
BUFFER_SIZE = 1000

dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

sequences = dataset.batch(SEQ_LENGTH + 1, drop_remainder=True)

def split_input_target(seq):
    return seq[:-1], seq[1:]

dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.Dense(vocab_size)
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
EPOCHS = 30   # increase later if needed

model.fit(dataset, epochs=EPOCHS)


Epoch 1/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 3.0326
Epoch 2/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 2.7896
Epoch 3/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 2.3872
Epoch 4/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 2.0002
Epoch 5/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 1.6702
Epoch 6/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 1.3747
Epoch 7/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 1.1193
Epoch 8/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.9051
Epoch 9/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.7361
Epoch 10/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.6057
Epoch 11/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.5082
Epoch 12/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.4368
Epoch 13/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.3834
Epoch 14/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.3432
Epoch 15/30
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.3123
Epoc

In [11]:
def generate_text(model, start_string, num_chars=300, temperature=0.7):
    input_indices = [char2idx[c] for c in start_string]
    input_indices = tf.expand_dims(input_indices, 0)

    generated_text = start_string

    for _ in range(num_chars):
        predictions = model(input_indices)
        predictions = predictions[:, -1, :] / temperature

        predicted_id = tf.random.categorical(predictions, 1)[0, 0].numpy()

        generated_text += idx2char[predicted_id]
        input_indices = tf.expand_dims([predicted_id], 0)

    return generated_text


In [12]:
print(generate_text(model, start_string="dear diary\n", num_chars=400, temperature=0.6))


dear diary
ti it molinecganghane arnwig t dwi t wathaterirog qineved isloneritis
lit atht wrneandane peay wres s s d t lithe me iry futhangal w

acamea methe llisthar


s
iti vealit t athes ane howakinsthe med t uthali wited me ut wri wrne sthat me mes t
lit it in mes meaveamy it itin t ly bean tithes ve lis i ml pe
me s ace qulomes pri an
iswr
me ine lme te titit tinbe t ups ane t ulowririt s the bes meat a


In [ ]:
#Task 5 report
# Handwritten Text Generation using Character-Level RNN

## Introduction
Handwritten text generation is a deep learning application where a model learns to generate human-like handwritten text patterns. In this project,
a character-level Recurrent Neural Network (RNN) using LSTM is implemented to generate handwritten-style text from a given dataset.

## Objective
- To implement a character-level RNN using TensorFlow  
- To train the model on handwritten-style text  
- To generate new handwritten-like text  

## Dataset
The dataset consists of handwritten-style diary text stored in a text file. It contains lowercase letters, spaces, and punctuation to
resemble natural handwriting. The dataset is repeated to increase training data.

## Methodology
- Load and preprocess text data  
- Encode characters into numerical values  
- Create fixed-length input sequences  
- Train an LSTM-based RNN model  
- Generate text using a seed string  

## Model Used
- Embedding Layer  
- LSTM Layer  
- Dense Output Layer  

## Result
The trained model successfully generates handwritten-style text that follows the writing pattern, flow, and style of the input dataset.

## Conclusion
This project demonstrates that a character-level RNN can effectively learn handwritten-style text patterns and generate realistic text outputs
using deep learning techniques.
